In [1]:
from config import FORVO_API_KEY
import asyncio
import aiohttp
import urllib.parse
import pandas as pd

In [2]:
df = pd.read_csv('basic_french_flashcards.csv', sep="&")
records = df.to_dict('records')

In [3]:
async def get_sound_file(dictionary, session, api_key = FORVO_API_KEY,):
    word = urllib.parse.quote(dictionary["french_word"])
    freq_idx = dictionary["frequency_idx"]
    url =  f"https://apifree.forvo.com/key/{api_key}/format/json/action/standard-pronunciation/word/{word}/language/fr"
    file_name = f"french_audio_{freq_idx}.mp3"
    path = "mp3_files/"
    try:
        async with session.get(url) as response:
            json_response = await response.json()
            mp3_download_url = json_response["items"][0]["pathmp3"]
        async with session.get(mp3_download_url) as mp3_response:
            mp3_file = await mp3_response.read()
        if sys.getsizeof(mp3_file) < 100:
            raise ValueError('No File Downloaded')
        with open(f'{path}{file_name}', 'wb') as local_file:
            local_file.write(mp3_file)
            print(f'Downloaded {freq_idx}')
    except Exception as e:
        return (dictionary, str(e))
    


async def get_multiple_sound_files(match_dict):
    all_responses = []
    async with aiohttp.ClientSession() as session:
        chunked_match_dicts = [match_dict[i:i + 5] for i in range(0, len(match_dict), 5)]
        for chunked_match_dict in chunked_match_dicts:
            tasks = []
            tasks.append(asyncio.sleep(5))
            for item in chunked_match_dict:
                tasks.append(get_sound_file(item, session))
            responses = await asyncio.gather(*tasks)
            all_responses.extend([response for response in responses if response is not None])
    return all_responses

In [4]:
failed_downloads = await get_multiple_sound_files(records[2630:2880])

Downloaded 2631
Downloaded 2634
Downloaded 2633
Downloaded 2632
Downloaded 2635
Downloaded 2640
Downloaded 2639
Downloaded 2638
Downloaded 2637
Downloaded 2636
Downloaded 2642
Downloaded 2641
Downloaded 2644
Downloaded 2645
Downloaded 2643
Downloaded 2647
Downloaded 2648
Downloaded 2646
Downloaded 2650
Downloaded 2649
Downloaded 2651
Downloaded 2654
Downloaded 2653
Downloaded 2655
Downloaded 2652
Downloaded 2656
Downloaded 2659
Downloaded 2660
Downloaded 2657
Downloaded 2658
Downloaded 2661
Downloaded 2665
Downloaded 2664
Downloaded 2662
Downloaded 2663
Downloaded 2669
Downloaded 2670
Downloaded 2667
Downloaded 2668
Downloaded 2666
Downloaded 2673
Downloaded 2672
Downloaded 2674
Downloaded 2671
Downloaded 2675
Downloaded 2677
Downloaded 2676
Downloaded 2678
Downloaded 2679
Downloaded 2680
Downloaded 2681
Downloaded 2682
Downloaded 2683
Downloaded 2684
Downloaded 2685
Downloaded 2686
Downloaded 2687
Downloaded 2690
Downloaded 2689
Downloaded 2688
Downloaded 2693
Downloaded 2691
Download